In [1]:
import urllib2
from bs4 import BeautifulSoup as soup
import numpy as np
import pandas as pd
import re
import time
import sys
import pickle
import os

In [2]:
url = 'https://www.monster.com/jobs/browse/q-it-jobs.aspx'
#Sample Job link
url_job = 'https://www.monster.com/jobs/q-it-operations-manager-jobs.aspx' 

In [3]:
# Get the list of Job and its link from the page: url = 'https://www.monster.com/jobs/browse/q-it-jobs.aspx'
def get_JobLink(url):
    try:
        page = urllib2.urlopen(url, timeout = 1)
    except urllib2.URLError, e:
        raise MyException("There was an error: %r" % e)
    else:
        s = soup(page, "html.parser")
        tags = s.find('ul','card-columns browse-all')
        links = tags.find_all('a')
    
    Job = list()
    Link = list()
    for link in links:
        if(link.text !='Select State'):
            j = link.text
            l = link['href']
        #print 'Job: %s, Link: %s' %(j,l)
            Job.append(j.replace(' Jobs',""))
            Link.append(l)
    Job_ = pd.DataFrame({'job':Job,'link':Link})
    return Job_

In [4]:
def get_Pages(links):
    
    Pages = list()
    
    for link in links:
        if link != "":
            try:
                page = urllib2.urlopen(link, timeout = 3)
            except urllib2.HTTPError:
                Pages.append('Error: 404')
            except urllib2.URLError, e:
                Pages.append('Error: Timeout 3')
            except Exception:
                Pages.append('Error: Exception')
            else:
                s = soup(page, "html.parser")
                Pages.append(s)
        else:    
            Pages.append('Error: Blank URL')
                         
    return Pages

In [5]:
# extracting the Number of Posting Info
def get_NumJob(s): # input: soup
    if isinstance(s,soup):
        t = s.find('div', attrs={"class":'col-xs-12 jsresultsheader'}).h2.text
        p = re.search(re.compile(' \d+( |\+)'),t)
        num = t[p.start():p.end()-1]
        num = int(num.strip())
    else:
        num = np.nan
    return num

In [6]:
def get_Nums(pages): #pages of beautifulsoup
    Num = list()
    for i in range(len(pages)):
        #print i
        num = get_NumJob(pages[i])
        Num.append(num)
    return Num

In [7]:
# Extracting the City and the State information of a Post
def get_City_State(s, job_title): # input: soup
    City = list()
    State = list()
    Link = list()
    Job = list()
    
    def add_item(city, state, url, job_title = job_title):
        City.append(city)
        State.append(state)
        Link.append(url)
        Job.append(job_title)
        
    
    if isinstance(s,soup):
        tags = s.find('li',attrs={"id":'backlinkingwidgetCity'})
        try:
            links = tags.find_all('a')
        except AttributeError:
            add_item(city = "", state = "", url = "")
        else:
            for link in links:
                if("title" in link.attrs):
                    t = link['title']
                    p = re.search(re.compile(',\s*([^,]+)$'),t)
                    state = t[p.start()+2:len(t)]
                    city = link.text
                    url = link['href']

                    add_item(city, state, url)
    else:
        add_item(city = "", state = "", url = "")
        
    Location_ = pd.DataFrame({'job':Job, 'city':City,'state':State, 'link': Link})
    return Location_

In [8]:
def get_Job_Location(pages,jobs): #input: list of Pages and the respective Job Titles
    Job_Location = pd.DataFrame()
    for i in range(len(pages)):
        job_loc = get_City_State(pages[i],jobs[i])
        Job_Location = pd.concat([Job_Location,job_loc], ignore_index=True)
        #print i
    return Job_Location

In [9]:
# extracting the Posting time information
def get_Job_PostTime(s, job, state, city): # input: soup
    Time = list()
    Link = list()
    Job = list()
    Title = list()
    State = list()
    City = list()
    
    def add_item(title, time, url, job = job, state = state, city = city):
        Title.append(title)
        Time.append(time)
        Link.append(url)
        Job.append(job)
        State.append(state)
        City.append(city)
        
    
    if isinstance(s,soup):
        tags = s.find_all('article',attrs={"class":'js_result_row'})
        for tag in tags:
            time = tag.time['datetime']
            url = tag.find('a')['href']
            title = tag.find('a')['title']
 
            add_item(title, time, url)
    else:
        add_item(title = "", time = "", url = "")
        
    PostTime_ = pd.DataFrame({'job':Job, 'title':Title,'time':Time, 'link': Link, 'state': State, 'city': City})
    return PostTime_

In [10]:
def get_Job_Time(pages,jobs,states,cities): #input: list of Pages of Location and the respective Job_Location.job
    Job_Time = pd.DataFrame()
    for i in range(len(pages)):
        job_time = get_Job_PostTime(pages[i],jobs.values[i],states.values[i],cities.values[i])
        Job_Time = pd.concat([Job_Time,job_time], ignore_index=True)
        #print i
    return Job_Time

In [11]:
def save_to_pickle(data, name):
    if isinstance(data,list):
        with open(name, 'wb') as f:
            pickle.dump(data, f)
    elif isinstance(data, pd.DataFrame) or isinstance(data, pd.Series):
        data.to_pickle(name)
    else:
        return "Not valid Data Type. Neither List, nor Pandas Object"

In [12]:
def get_from_pickle(pickle_file_name):
    
    #Restoring Objects from Pickle files
    with open(pickle_file_name) as f:
        loaded_obj = pickle.load(f)
    
    #Restoring DataFrame Objects from Pickle files
    # df2 = pd.read_pickle('my_df.pickle')
    
    return loaded_obj

In [19]:
#Page Level 1 = IT Jobs Home page with list of Job Titles
#Page Level 2 = Job Title Home Page with list of Postings for the selected Title
#Page Level 3 = List of Jobs on Location with list of Postings for the selected Title and Location

def extract_from_P3(Jobs_Location):
    NumL = [0]*len(Jobs_Location)
    Job_Time = pd.DataFrame()
    files = filter(lambda x: re.match("webscrapping_Pages_Location",x), os.listdir('./'))
    
    for file_ in files:       
        Pages = get_from_pickle(file_)
        
        n = int(re.search(re.compile('\d+'),file_).group())-1
        from_ = n*200
        to_ = from_ + len(Pages)
        Jobs = Jobs_Location.job[from_:to_]
        States = Jobs_Location.state[from_:to_]
        Cities = Jobs_Location.city[from_:to_]
        
        job_time = get_Job_Time(Pages,Jobs,States,Cities)
        Job_Time = pd.concat([Job_Time,job_time], ignore_index=True)
        
        NumL[from_:to_] = get_Nums(Pages)
        
        print n
        
    return Job_Time, NumL

In [14]:
def download_Pages_toFile(Jobs_Location, step=200, get_all=False):
    n = 0
    from_ = n*step
    to_ = from_ + step
    
    while from_ < len(Jobs_Location.link):
        to_ = to_ if to_< len(Jobs_Location.link) else len(Jobs_Location.link)
        links = Jobs_Location.link[from_:to_]
        Pages = get_Pages(Jobs_Location.link)
        
        file_name = "webscrapping_Pages_Location" + "_" + str(n) + ".pickle"
        save_to_pickle(Pages_Location,file_name)
        
        if(get_all):
            n = n+1
            from_ = n*200
            to_ = from_ + step
        else:
            from_ = len(Jobs_Location.link) + 1    

In [15]:
def extract_Num_Location(Jobs_Location):
    NumL = [0]*len(Jobs_Location)
    #Job_Time = pd.DataFrame()
    files = filter(lambda x: re.match("webscrapping_Pages_Location",x), os.listdir('./'))
    
    for file_ in files:       
        Pages = get_from_pickle(file_)
        
        n = int(re.search(re.compile('\d+'),file_).group())-1
        from_ = n*200
        to_ = from_ + len(Pages)
        #Jobs = Jobs_Location.job[from_:to_]
        
        #job_time = get_Job_Time(Pages,Jobs)
        #Job_Time = pd.concat([Job_Time,job_time], ignore_index=True)
        
        NumL[from_:to_] = get_Nums(Pages)
        
        print n
        
    return NumL

In [10]:
IT_Jobs = get_JobLink(url)

In [ ]:
# Scrapping Pages of IT Jobs
Pages_of_IT_Jobs = get_Pages(IT_Jobs.link)

In [13]:
# Saving the IT Jobs Pages to file
save_to_pickle(Pages_of_IT_Jobs,"webscrapping_Pages_of_IT_Jobs.pickle")

In [34]:
# Extracting the Num of Job Posts on each IT Job Title Page
NumJ = get_Nums(Pages_of_IT_Jobs)

# Adding this number of Job Posts on each IT Job Title to the IT_Jobs DataFrame
IT_Jobs['count'] = NumJ

In [52]:
# Saving IT Jobs DataFrame to file
save_to_pickle(IT_Jobs,"webscrapping_IT_Jobs_df.pickle")

In [10]:
#IT_Jobs = get_from_pickle("webscrapping_IT_Jobs_df.pickle")

In [ ]:
Jobs_Location = get_Job_Location(Pages_of_IT_Jobs, IT_Jobs.job)

In [16]:
#Jobs_Location = get_from_pickle("webscrapping_Jobs_Location_df.pickle")

In [ ]:
download_Pages_P3(Jobs_Location, get_all=True)

In [20]:
data = extract_from_P3(Jobs_Location)

0
9
10
11
12
13
14
15
16
17
18
1
19
20
21
22
23
24
25
26
27
28
2
29
30
31
32
33
34
35
36
37
3
4
5
6
7
8


In [21]:
NumL = data[1]
Job_Time = data[0]

In [128]:
Job_Location['count'] = NumL

In [167]:
save_to_pickle(Jobs_Location, "webscrapping_Jobs_Location_df.pickle")

In [ ]:
# Save Job_Time into File
save_to_pickle(Job_Time, "webscrapping_Job_Time_df.pickle")

In [4]:
Pages_Location = get_from_pickle("webscrapping_Pages_of_IT_Jobs.pickle")

In [6]:
len(Pages_Location)

882

In [96]:
Job_Location = get_Job_Location(Pages_Location, IT_Jobs.job)

In [129]:
Job_Location.to_csv('webscrapping_Jobs_Location_df.csv', encoding = "utf-8")

In [23]:
Job_Time.to_csv('webscrapping_Job_Time_df.csv', encoding = "utf-8")